# Setup

In [2]:
from dotenv import load_dotenv
env = load_dotenv()

In [3]:
from langgraph.graph import StateGraph, END
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_core.output_parsers import JsonOutputParser
from langchain_experimental.llms.ollama_functions import OllamaFunctions
from langchain_openai import ChatOpenAI
from IPython.display import Image
import IPython

# Tools

In [4]:
from typing import TypedDict, Annotated, Dict, Any
import operator

class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]

In [7]:
%pip install pypdf

Note: you may need to restart the kernel to use updated packages.


In [32]:
from langchain.tools import tool
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_experimental.text_splitter import SemanticChunker
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain.pydantic_v1 import BaseModel, Field
from pinecone import Pinecone, ServerlessSpec
from bs4 import BeautifulSoup
from PyPDF2 import PdfReader
import tiktoken
import requests

# Search for relevant menu
search_tool = TavilySearchResults()

# Scrape menu data
class ScrapeInput(BaseModel):
    url: str = Field(description="the URL of the menu page")
    url: str = Field(description="the URL of the menu page")

@tool("scrape_pdf", args_schema=ScrapeInput, return_direct=True)
def scrape_pdf(url: str):
    """Scrape a webpage that may include links to a restaurants current menu and return the links"""
    pdf_links = []
    try:
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        links = soup.find_all('a')
        pdf_links = [link.get('href') for link in links if link.get('href').endswith('.pdf')]
    except Exception as e:
        print(f"failed to scrape {url} ERROR: {e}")
    return pdf_links

@tool("scrape_text", args_schema=ScrapeInput, return_direct=True)
def scrape_text(url: str):
    """Scrape the text directly from a website"""
    try:
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        # Remove script and style elements
        for script in soup(["script", "style"]):
            script.decompose()
        # Get text
        text = soup.get_text(separator=' ')
        # Break into lines and remove leading and trailing space on each
        lines = (line.strip() for line in text.splitlines())
        # Break multi-headlines into a line each
        chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
        # Drop blank lines
        text = '\n'.join(chunk for chunk in chunks if chunk)
    except Exception as e:
        print(f"failed to scrape {url} ERROR: {e}")
        return None
    return text

class SearchInput(BaseModel):
    download_url: str = Field(description="the URL of the menu pdf")
    name: str = Field(description="the name of the restaurants")

@tool("download-pdf", args_schema=SearchInput, return_direct=True)
def download_pdf(download_url: str, filename: str):
    """Download a pdf file from a given url and filename"""
    response = urlopen(download_url)
    file = open("pdf/"+filename+".pdf", 'wb')
    file.write(response.read())
    file.close()

def read_pdf(file_path) -> str:
    output = []
    try:
        with open(file_path, 'rb') as file:
            pdf_reader = PdfReader(file)
            for page in pdf_reader.pages:
                output.append(page.extract_text())

        return output
        
    except Exception as e:
        print(f"Error reading PDF file: {e}")

# Tokenize the pdf 
class upsertInput(BaseModel):
    pdf_url: str = Field(description="The url of the PDF file that must be upserted")


@tool("tokenize-pdf", args_schema=upsertInput, return_direct=True)
def upsert_pdf(file_path):
    """   """
    # Split the input text 
    text_splitter_semantic = SemanticChunker(
        OpenAIEmbeddings(), breakpoint_threshold_type="percentile"
    )

    file = read_pdf(file_path)
    docs = text_splitter_semantic.create_documents(file)

    # embeddings = OpenAIEmbeddings()

    # Upsert to the Pinecone datastore
    pc = Pinecone()
    index = pc.Index("paprika-ragu")

    
    
    # Save to vector DB with formatted metada
    pass

In [28]:
from PyPDF2 import PdfReader

def read_pdf(file_path) -> str:
    output = []
    try:
        with open(file_path, 'rb') as file:
            pdf_reader = PdfReader(file)
            for page in pdf_reader.pages:
                output.append(page.extract_text())

        return output
        
    except Exception as e:
        print(f"Error reading PDF file: {e}")

# Replace 'your_pdf_file.pdf' with the path to your PDF file
file = read_pdf('pdf/menu.pdf')

In [8]:
from langchain_community.document_loaders import PyPDFLoader

def read_pdf(file_path) -> str:
    loader = PyPDFLoader(file_path)
    pages = loader.load()
    return pages
    
file = read_pdf('pdf/menu.pdf')

In [29]:
print(file)

["CRISP\nRochester\nVegan Meatloaf ......................................................................................................$28\nCarrot puree and haricots verts. *Vegan *GF\nPan Roasted Skin-on Salmon.................................................................................$28\nTri-color tomatoes, fingerlings, asparagus, fried capers. *GF\nHeaven in a Bowl.....................................................................................................$27\nFresh garganelli, baby spinach, asparagus, zucchini, tomatoes, pesto,\ntoasted pine nuts. \nBuffalo Chicken Mac ‘n Cheese .............................................................................$20\nFresh cavatappi, cheddar, jack, parmesan, buffalo-popcorn chicken, \nhouse blue cheese and crumbles. \nDuck Confit..............................................................................................................$32\nMashed yams, sautéed kale, goat cheese crumbles, candied pecans, sour\ncherry gastr

In [20]:
from langchain_experimental.text_splitter import SemanticChunker
from langchain_openai.embeddings import OpenAIEmbeddings

# Split the input text 
text_splitter_semantic = SemanticChunker(
    OpenAIEmbeddings(), breakpoint_threshold_type="percentile"
)

In [21]:
split_text_sem = text_splitter_semantic.create_documents(file)

In [22]:
split_text_sem

[Document(page_content='CRISP\nRochester\nVegan Meatloaf ......................................................................................................$28\nCarrot puree and haricots verts. *Vegan *GF\nPan Roasted Skin-on Salmon.................................................................................$28\nTri-color tomatoes, fingerlings, asparagus, fried capers. *GF\nHeaven in a Bowl.....................................................................................................$27\nFresh garganelli, baby spinach, asparagus, zucchini, tomatoes, pesto,\ntoasted pine nuts. Buffalo Chicken Mac ‘n Cheese .............................................................................$20\nFresh cavatappi, cheddar, jack, parmesan, buffalo-popcorn chicken, \nhouse blue cheese and crumbles. Duck Confit..............................................................................................................$32\nMashed yams, sautéed kale, goat cheese crumbles, candied pecans, 

In [82]:
len(split_text_sem)

4

In [88]:
split_text_sem[3]

Document(page_content="*V\nS. Clinton Ave. Tacos \nServed with house tortilla chips, salsa, and guacamole. Choice of:\nCarnitas ...........................................................................................................$17  \nAja verde, white onions, cilantro. Baja Fish ..........................................................................................................$17 \nSlaw, aja verde, Mexican crema, pickled red onions, cilantro, Cotija. Vegan ...............................................................................................................$18 \nSeasoned ground Impossible “meat”, pickled red onions, \ncilantro, hot sauce. *V\nBraised Beef and Carrots ..........................................................................$28\nBraised in a rich Pinot Noir laced pan sauce, garlic mashed potatoes, \nseasonal veggies, gremolata. Shrimp 'n Grits ............................................................................................$28\nCheesy g

In [83]:
len(split_text_rec)

50

In [15]:
tools = [search_tool, scrape_pdf, scrape_text, download_pdf, tokenize_pdf]

# RAG Model

In [16]:
class Model:
    # Define the model
    def __init__(self, model, tools, system=""):
        self.system = system
        graph = StateGraph(AgentState)
        graph.add_node("llm", self.openai_inference)
        graph.add_node("action", self.take_action)
        graph.add_conditional_edges(
            "llm",
            self.exists_action,
            {True: "action", False: END}
        )
        graph.add_edge("action", "llm")
        graph.set_entry_point("llm")
        self.graph = graph.compile()
        self.tools = {t.name: t for t in tools}
        self.model = model.bind_tools(tools)

    # Check if llm requires action 
    def exists_action(self, state: AgentState):
        result = state['messages'][-1]
        return len(result.tool_calls) > 0
    # Run a tool ordered by the model
    def take_action(self, state: AgentState):
        tool_calls = state['messages'][-1].tool_calls
        results = []
        for t in tool_calls:
            print(f"Calling: {t}")
            result = self.tools[t['name']].invoke(t['args'])
            results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result)))
        print("Back to the model!")
        return {'messages': results}
    
    # Invokes the current message chain
    def openai_inference(self, state: AgentState):
        messages = state['messages']
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        message = self.model.invoke(messages)
        return {'messages': [message]}

In [17]:
prompt = """You are tasked with finding and storing the text content of a given restaurants Menu. 
The menu may be a PDF file that you will be required to scrape, or it will be text on a menu page. 
ONLY scrape menu data, this includes foods that the restaurant offers and DOES NOT include information about the restaurant itself or the menu iteself.
Use the search tool to find the url of the menu, if you cannot find the pdf url directly use the scrape_pdf tool to scrape webpages for the url.
If the menu isn't stored as a PDF, and is instead stored as text on a webpage, use the scrape_text tool to gather the text from the page. 
ONLY use the scrape_text tool if a pdf_menu is unavailable. DO NOT attempt to scrape blank menus. 
Once you have scraped the menu there are no more tasks to be done. 
"""

llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0,
    max_tokens=None,
    timeout=None,
)

model = Model(llm, tools, system=prompt)

In [43]:
messages = ["CRISP Rochester"]
result = model.graph.invoke({"messages": messages})

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'CRISP Rochester menu'}, 'id': 'call_sNkgSzYtU6VRa0GtWo32fbLr', 'type': 'tool_call'}
Back to the model!
Calling: {'name': 'scrape_pdf', 'args': {'url': 'https://crisprochester.com/menus/'}, 'id': 'call_j3WCuxRyU9Mu685xLKZR3t8q', 'type': 'tool_call'}
Calling: {'name': 'scrape_text', 'args': {'url': 'https://crisprochester.com/menus/'}, 'id': 'call_qSktkVOL9mKcTqjZ6YMFKpgz', 'type': 'tool_call'}
Back to the model!
Calling: {'name': 'scrape_pdf', 'args': {'url': 'https://crisprochester.com/wp-content/uploads/Lunch-and-Dinner.pdf'}, 'id': 'call_hY3A3nrSA8SezHLLskdBbBdl', 'type': 'tool_call'}
Calling: {'name': 'scrape_pdf', 'args': {'url': 'https://crisprochester.com/wp-content/uploads/Brunch.pdf'}, 'id': 'call_QazKxYNGlnmEMjk5MM47vh0d', 'type': 'tool_call'}
Calling: {'name': 'scrape_pdf', 'args': {'url': 'https://crisprochester.com/wp-content/uploads/Desserts.pdf'}, 'id': 'call_6ouNjZJBALApxevm8tjF4Fgk', 'type': 'tool_call'}

In [44]:
result

{'messages': ['CRISP Rochester',
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_sNkgSzYtU6VRa0GtWo32fbLr', 'function': {'arguments': '{"query":"CRISP Rochester menu"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 327, 'total_tokens': 348}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_3cd8b62c3b', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-21a3eed3-915e-4594-b771-b06d88708c2d-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'CRISP Rochester menu'}, 'id': 'call_sNkgSzYtU6VRa0GtWo32fbLr', 'type': 'tool_call'}], usage_metadata={'input_tokens': 327, 'output_tokens': 21, 'total_tokens': 348}),
  ToolMessage(content="[{'url': 'https://crisprochester.com/menus/', 'content': 'CRISP Rochester menus. Lunch, dinner, cocktails, wine, beer, and brunch menus. Skip to content. 819 S. Clinton Ave. Rochester, NY. 14620 +1 (585) 978-

In [17]:
import pygraphviz
Image(model.graph.get_graph().draw_png())

ModuleNotFoundError: No module named 'pygraphviz'

# Datastore

In [90]:
%pip install --upgrade --quiet  \
    langchain-pinecone \
    langchain-openai \
    langchain \
    langchain-community \
    pinecone-notebooks

Note: you may need to restart the kernel to use updated packages.


In [23]:
from PyPDF2 import PdfReader

def read_pdf(file_path) -> str:
    output = []
    try:
        with open(file_path, 'rb') as file:
            pdf_reader = PdfReader(file)
            for page in pdf_reader.pages:
                output.append(page.extract_text())

        return output
        
    except Exception as e:
        print(f"Error reading PDF file: {e}")

# Replace 'your_pdf_file.pdf' with the path to your PDF file
file = read_pdf('pdf/menu.pdf')

In [24]:
docs = text_splitter_semantic.create_documents(file)

In [25]:
embeddings = OpenAIEmbeddings()

In [27]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone()

In [23]:
import time

index_name = "paprika-ragu"

existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=1536,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )
    while not pc.describe_index(index_name).status["ready"]:
        time.sleep(1)

index = pc.Index(index_name)

In [24]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(docs, embeddings, index_name=index_name)

In [1]:
query = "Does CRISP Rochester fit a vegan diet?"
docs = docsearch.similarity_search(query)
print(docs[0].page_content)

NameError: name 'docsearch' is not defined